<center>
<a href="http://www.insa-toulouse.fr/" ><img src="http://www.math.univ-toulouse.fr/~besse/Wikistat/Images/logo-insa.jpg" style="float:left; max-width: 120px; display: inline" alt="INSA"/></a> 

<a href="http://wikistat.fr/" ><img src="http://www.math.univ-toulouse.fr/~besse/Wikistat/Images/wikistat.jpg" style="max-width: 250px; display: inline"  alt="Wikistat"/></a>


<a href="http://www.hupi.fr/" ><img src="http://www.hupi.fr/wp-content/uploads/2016/03/hupi_logo_vectoris_menu.png" style="float:right; max-width: 300px; display: inline"  alt="Wikistat"/></a>

 </a>
</center>

# [Ateliers: Technologies des grosses data](https://github.com/wikistat/Ateliers-Big-Data)

# [Reconnaissance de caractères manuscrits](https://github.com/wikistat/Ateliers-Big-Data/2-MNIST) ([MNIST](http://yann.lecun.com/exdb/mnist/)) avec <a href="http://spark.apache.org/"><img src="http://spark.apache.org/images/spark-logo-trademark.png" style="max-width: 100px; display: inline" alt="Spark"/> </a> et [SparkML](https://spark.apache.org/docs/latest/ml-statistics.html)

## Introduction

Même traitement en utilisant la librairie SparkML.

In [1]:
sc

<SparkContext master=local[*] appName=pyspark-shell>

In [2]:
# Importation des packages
import time
from numpy import array
# Répertoire courant ou répertoire accessible de tous les "workers" du cluster
DATA_PATH="/Users/bguillouet/Insa/TP_Insa/data/" 

## Gestion des données

### Importation et transformation des données au format RDD

Les données sont déjà partagée en une partie apprentissage et une test utilisée pour les comparaisons entre méthodes dans les publications. Ce sont bien les données du site MNIST mais transformée au format .csv pour en faciliter la lecture. 

Elles doivent être stockées à un emplacement accessibles de tous les noeuds du cluster pour permettre la construction de la base de données réparties (RDD). 

Dans une utilisation monoposte (*standalone*) de *Spark*, elles sont simplement chargées dans le répertoire courant. 

In [4]:
# Chargement des fichiers
import urllib.request
f = urllib.request.urlretrieve("https://www.math.univ-toulouse.fr/~besse/Wikistat/data/mnist_train.csv",DATA_PATH+"mnist_train.csv")
f = urllib.request.urlretrieve("https://www.math.univ-toulouse.fr/~besse/Wikistat/data/mnist_test.csv",DATA_PATH+"mnist_test.csv")

In [5]:
# Transformation du fichier texte en RDD de valeurs
## Données d'apprentissage
# Transformation ou étape map de séparation des champs
trainRDD = sc.textFile(DATA_PATH+"mnist_train.csv").map(lambda l: [float(x) for x in l.split(',')])
# Action
trainRDD.count() # taille de l'échantillon
#test

60000

### Conversion des données au format DataFrame

Pour pouvoir être intérprété par les différentes méthodes de classification de la librairie SparkML, les données doivent être converties en objet DataFrame.

Pour plus d'information sur l'utilisation de ces DataFrames, reportez vous aux calepins 1-Intro-PySpark/Cal2-PySpark-statelem-SparkML et 1-Intro-PySpark/Cal3-PySpark-SQL.ipynb

In [6]:
# Transformation du de la RDD en DataFrame

from pyspark.sql import Row
from pyspark.ml.linalg import Vectors

#Cette fonction va permettre de transformer chaque ligne de la RDD en une "Row" pyspark.sql. 

def list_to_Row(l):    
    #Creation d'un vecteur sparse pour les features
    features = Vectors.sparse(784,dict([(i,v) for i,v in enumerate(l[:-1]) if v!=0]))
    row = Row(label = l[-1], features= features)
    return row

trainDF = trainRDD.map(list_to_Row).toDF()

In [7]:
# Exemple de ligne
trainDF.take(1)[0]

Row(features=SparseVector(784, {118: 25.0, 119: 105.0, 120: 105.0, 121: 105.0, 122: 123.0, 123: 253.0, 124: 253.0, 125: 253.0, 126: 253.0, 127: 255.0, 128: 253.0, 129: 156.0, 130: 55.0, 146: 183.0, 147: 252.0, 148: 252.0, 149: 252.0, 150: 252.0, 151: 252.0, 152: 252.0, 153: 252.0, 154: 252.0, 155: 253.0, 156: 252.0, 157: 252.0, 158: 231.0, 159: 164.0, 160: 18.0, 174: 208.0, 175: 252.0, 176: 252.0, 177: 252.0, 178: 252.0, 179: 252.0, 180: 252.0, 181: 252.0, 182: 252.0, 183: 253.0, 184: 252.0, 185: 252.0, 186: 252.0, 187: 252.0, 188: 74.0, 202: 110.0, 203: 210.0, 204: 252.0, 205: 216.0, 206: 132.0, 207: 132.0, 208: 132.0, 209: 132.0, 210: 132.0, 211: 133.0, 212: 160.0, 213: 252.0, 214: 252.0, 215: 252.0, 216: 143.0, 231: 19.0, 232: 29.0, 233: 20.0, 240: 60.0, 241: 252.0, 242: 252.0, 243: 252.0, 244: 178.0, 267: 13.0, 268: 170.0, 269: 252.0, 270: 252.0, 271: 243.0, 272: 65.0, 295: 183.0, 296: 252.0, 297: 252.0, 298: 252.0, 299: 177.0, 322: 53.0, 323: 253.0, 324: 252.0, 325: 252.0, 326: 20

In [8]:
## Même chose pour les données de test
testRDD = sc.textFile(DATA_PATH+'mnist_test.csv').map(lambda l: [float(x) for x in l.split(',')])
testRDD.count() # taille de l'échantillon

10000

In [9]:
testDF = testRDD.map(list_to_Row).toDF()
testDF.take(1)

[Row(features=SparseVector(784, {151: 5.0, 152: 127.0, 153: 231.0, 154: 194.0, 155: 83.0, 178: 35.0, 179: 203.0, 180: 253.0, 181: 253.0, 182: 237.0, 183: 237.0, 184: 199.0, 185: 6.0, 187: 53.0, 188: 53.0, 206: 224.0, 207: 253.0, 208: 228.0, 209: 123.0, 210: 18.0, 211: 89.0, 212: 247.0, 213: 54.0, 214: 13.0, 215: 213.0, 216: 236.0, 217: 27.0, 233: 91.0, 234: 254.0, 235: 242.0, 236: 43.0, 239: 50.0, 240: 159.0, 242: 182.0, 243: 253.0, 244: 244.0, 245: 33.0, 261: 98.0, 262: 254.0, 263: 233.0, 269: 99.0, 270: 248.0, 271: 253.0, 272: 120.0, 289: 38.0, 290: 255.0, 291: 246.0, 292: 48.0, 296: 80.0, 297: 254.0, 298: 254.0, 299: 133.0, 317: 46.0, 318: 254.0, 319: 253.0, 320: 226.0, 321: 72.0, 322: 95.0, 323: 140.0, 324: 232.0, 325: 253.0, 326: 253.0, 327: 58.0, 345: 68.0, 346: 254.0, 347: 253.0, 348: 253.0, 349: 253.0, 350: 253.0, 351: 254.0, 352: 253.0, 353: 253.0, 354: 210.0, 355: 40.0, 374: 19.0, 375: 102.0, 376: 211.0, 377: 253.0, 378: 253.0, 379: 254.0, 380: 227.0, 381: 49.0, 382: 9.0, 404

### Sous-échantillon d'apprentissage

Extraction d'un sous-échantillon d'apprentissage pour tester les programmes sur des données plus petites. Itérer cette démarche permet d'étudier l'évolution de l'erreur de prévision en fonction de la taille de l'échantillon d'apprentissage.

In [10]:
tauxEch=0.1 # tester pour des tailles croissantes d'échantillon d'apprentissage
(trainData, DropDatal) = trainRDD.randomSplit([tauxEch, 1-tauxEch])
trainData.count()

6020

## Méthode de classification

Les méthodes de classifications de la librairie SparkML suivent le même shéma d'utilisation. 

Il faut dans un premier temps crée un objets **methode** pour configurer les paramètres de la méthode.
Dans un second temps on réalise l'apprentissage en appliquant la fonction **fit** de la méthode sur la DataFrame d'apprentissage. Cette commande créé un objet différent, le **model** qui permettra de réaliser les prédictions. 

Par défaut les différentes méthodes considère que les noms des colonnes correspondants aux variables et au prédicants du jeux d'apprentissage sont respectivement "features" et "label". Tandis que les prédictions seront automatiquement assigné à une colonne de nom "prediction". 
Il est conseillé de garder cette terminiologie, mais ces attributs par défaut peuvent être modifié en spécifiant les paramètres  *featuresCol*, *labelCol* et *predictionCol* de chaque méthode.

### Régression logistique

Exemple d'utilisation pour expliciter la syntaxe mais sans grand intérêt pour ces données qui ne satisfont pas à des frontières de discrimination linéaires. L'algorithme permettant de réaliser une regression logistique multinomial est l'algorithme [*softmax*](https://spark.apache.org/docs/latest/ml-classification-regression.html#multinomial-logistic-regression).

In [11]:
### Logistic Regression
from pyspark.ml.classification import LogisticRegression

### Configuraiton des paramètres de la méthode
time_start=time.time()
lr = LogisticRegression(maxIter=100, regParam=0.01, fitIntercept=False, tol=0.0001,
            family = "multinomial", elasticNetParam=0.0) #0 for L2 penalty, 1 for L1 penalty

### Génération du modèle
model_lr = lr.fit(trainDF)
 
time_end=time.time()
time_lrm=(time_end - time_start)
print("LR prend %d s" %(time_lrm)) # (104s avec taux=1)

LR prend 20 s


#### Erreur sur l'échantillon test

In [12]:
predictionsRDD = model_lr.transform(testDF)
labelsAndPredictions = predictionsRDD.select("label","prediction").collect()
nb_good_prediction = sum([r[0]==r[1] for r in labelsAndPredictions])
nb_test = testDF.count()
testErr = 1-nb_good_prediction/nb_test
print('Test Error = ' + str(testErr)) # (0.08 avec taux =1)

Test Error = 0.07789999999999997


In [ ]:
#LogisticRegressionTrainingSummary provides a summary for a LogisticRegressionModel. Currently, only binary classification is supported. Support for multiclass model summaries will be added in the future.

###  Arbre binaire de décision

Même chose pour un arbre de discrimination. Comme pour l'implémentation de scikit-learn, les arbres ne peuvent être optimisés par un élagage basé sur une pénalisation de la complexité. Ce paramètre n'est pas présent, seule la profondeur max ou le nombre minimal d'observations par feuille peut contrôler la complexité. Noter l'apparition d'un nouveau paramètre: *maxBins* qui, schématiquement, rend qualitative ordinale à maxBins classes toute variable quantitative.  D'autre part, il n'y a pas de représentation graphique. Cette implémentation d'arbre est issue d'un [projet Google](http://static.googleusercontent.com/media/research.google.com/fr//pubs/archive/36296.pdf) pour adapter cet algorithme aux contraintes *mapreduce* de données sous Hadoop. Elle vaut surtout pour permettre de construire une implémentation des forêts aléatoires.

In [13]:
### Decision Tree
from pyspark.ml.classification import DecisionTreeClassifier

### Configuraiton des paramètres de la méthode
time_start=time.time()
dt = DecisionTreeClassifier(impurity='gini',maxDepth=5,maxBins=32, minInstancesPerNode=1,
                            minInfoGain=0.0)

### Génération du modèle
model_dt = dt.fit(trainDF)

time_end=time.time()
time_dt=(time_end - time_start)
print("DT takes %d s" %(time_dt))

DT takes 46 s


#### Erreur sur l'échantillon test

In [14]:
predictionsRDD = model_dt.transform(testDF)
labelsAndPredictions = predictionsRDD.select("label","prediction").collect()
nb_good_prediction = sum([r[0]==r[1] for r in labelsAndPredictions])
nb_test = testDF.count()
testErr = 1-nb_good_prediction/nb_test
print('Test Error = ' + str(testErr)) 

Test Error = 0.3228


### Random Forest

Les $k$-nn ne sont pas "scalables" et donc pas présents. Voici la syntaxe et les paramètres associés à l'algorithme des forêts aléatoires. Parmi ceux "classiques" se trouvent *numTrees*, *featureSubsetStrategy*, *impurity*, *maxdepth* et en plus *maxbins* comme pour les arbres. Les valeurs du paramètres *maxDepth* est critique pour la qualité de la prévision. en principe, il n'est pas contraint, un arbre peut se déployer sans "limite" mais face à des données massives cela peut provoquer des plantages intempestifs.

In [15]:
### Random Forest
from pyspark.ml.classification import RandomForestClassifier

### Configuraiton des paramètres de la méthode
time_start=time.time()
rf = RandomForestClassifier(numTrees = 2, impurity='gini', maxDepth=12,
                            maxBins=32, seed=None)

### Génération du modèle
model_rf = rf.fit(trainDF)

time_end=time.time()
time_rf=(time_end - time_start)
print("RF takes %d s" %(time_rf))#

RF takes 48 s


#### Erreur sur l'échantillon test

Erreur sur l'échantillon test

In [16]:
predictionsRDD = model_rf.transform(testDF)
labelsAndPredictions = predictionsRDD.select("label","prediction").collect()
nb_good_prediction = sum([r[0]==r[1] for r in labelsAndPredictions])
nb_test = testDF.count()
testErr = 1-nb_good_prediction/nb_test
print('Test Error = ' + str(testErr)) 

Test Error = 0.15480000000000005


Même traitement sur la totalité de l'échantillon

## 3 Quelques résultats

100 arbres, sélection automatique, maxDepth=9

maxBins | Temps |  Erreur 
--------|-------|---------
32 | 259 |  0.067 
64 | 264 |  0.068 
128 | 490 | 0.065

100 arbres, sélection automatique, maxBins=32

maxDepth | Temps | Erreur
---------|-------|-------
4 | 55 | 0.21
9 | 259 |  0.067
18 | 983 | **0.035**

Le nombre de variables tirées à chaque noeud n'a pas été optimisé. 

Le paramètre maxBins ne semble pas trop influencer la précision du modèle, au contriare de la profondeur maximum des arbres. Avec une profondeur suffisante, on retrouve (presque) les résultats classiques des forêts aléatoires sur ces données.

COmparer les résultats obtenus pour les trois environnements. 